In [0]:
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# P100: 16GB (Works)
# T4: 16GB (Not tested)
# P4: 8GB (Not tested)
# K80: 8GB (Not tested)

!nvidia-smi -L

# **DAIN**

## **`PREPARE VM`**

---



In [0]:
def audio_notification():
  # Play an audio beep. Any audio URL will do.
  from google.colab import output
  for x in range(1, 4):
    output.eval_js('new Audio("https://notificationsounds.com/notification-sounds/maybe-one-day-584/download/mp3").play()')
  print("Done!")
print("OK!")

### Downgrade gcc

In [0]:
%%writefile /etc/apt/sources.list
# See http://help.ubuntu.com/community/UpgradeNotes for how to upgrade to
# newer versions of the distribution.
deb http://archive.ubuntu.com/ubuntu/ bionic main restricted
deb http://archive.ubuntu.com/ubuntu/ bionic-updates main restricted
deb http://archive.ubuntu.com/ubuntu/ bionic universe
deb http://archive.ubuntu.com/ubuntu/ bionic-updates universe
deb http://archive.ubuntu.com/ubuntu/ bionic multiverse
deb http://archive.ubuntu.com/ubuntu/ bionic-updates multiverse
deb http://archive.ubuntu.com/ubuntu/ bionic-backports main restricted universe multiverse
deb http://security.ubuntu.com/ubuntu/ bionic-security main restricted
deb http://security.ubuntu.com/ubuntu/ bionic-security universe
deb http://security.ubuntu.com/ubuntu/ bionic-security multiverse
deb https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/
deb http://dk.archive.ubuntu.com/ubuntu/ xenial main
deb http://dk.archive.ubuntu.com/ubuntu/ xenial universe

In [0]:
!gcc -dumpversion
!g++ -dumpversion

In [0]:
!sudo apt update
!sudo apt install gcc-4.9
!sudo apt install g++-4.9

# Remove & overwrite any existing symbolic links to activate freshly installed gcc

!rm /usr/bin/gcc
!rm /usr/bin/g++

!ln -s /usr/bin/gcc-4.9 /usr/bin/gcc
!ln -s /usr/bin/g++-4.9 /usr/bin/g++

!gcc -dumpversion
!g++ -dumpversion

### Downgrade Torch version

In [0]:
!yes | pip uninstall torch torchvision

!pip install torch==1.0.0 torchvision==0.2.1

import torch
print(torch.__version__)

audio_notification()

In [0]:
import torch
print(torch.__version__)

### Downgrade to Cuda 9.0

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1
!echo ****** Cuda reinstall completed. Restarting runtime now! *******

audio_notification()

import os
os.kill(os.getpid(), 9)



In [0]:
!nvcc --version

## **`PREPARE DAIN`**

---



### Download Repository


In [0]:
# Download repository
!git clone https://github.com/BrokenSilence/MVIMP.git

### Adjust the default nvcc_args

In [0]:
%%writefile /content/MVIMP/third_party/DAIN/my_package/DepthFlowProjection/setup.py

#!/usr/bin/env python3
import os
import torch

from setuptools import setup, find_packages
from torch.utils.cpp_extension import BuildExtension, CUDAExtension

cxx_args = ['-std=c++11']
nvcc_args = [
    #'-gencode', 'arch=compute_30,code=sm_30', #K80
    #'-gencode', 'arch=compute_37,code=sm_37', #K80
    #'-gencode', 'arch=compute_50,code=sm_50',
    #'-gencode', 'arch=compute_52,code=sm_52',
    '-gencode', 'arch=compute_60,code=sm_60',  #P100
    #'-gencode', 'arch=compute_61,code=sm_61'  #P4
    # '-gencode', 'arch=compute_70,code=sm_70',
    #'-gencode', 'arch=compute_70,code=compute_70'
]

setup(
    name='depthflowprojection_cuda',
    ext_modules=[
        CUDAExtension('depthflowprojection_cuda', [
            'depthflowprojection_cuda.cc',
            'depthflowprojection_cuda_kernel.cu'
        ], extra_compile_args={'cxx': cxx_args, 'nvcc': nvcc_args})
    ],
    cmdclass={
        'build_ext': BuildExtension
    })

### Build DAIN

In [0]:
# If you played around uninstall previously build/installed packages
!pip uninstall mindepthflowprojection_cuda flowprojection_cuda separableconv_cuda depthflowprojection_cuda interpolationch_cuda interpolation_cuda separableconvflow-cuda filterinterpolation_cuda correlation_cuda -y

In [0]:
# This takes a while. Just wait. ~15 minutes.
# Building DAIN.
%cd /content/MVIMP/third_party/DAIN/my_package/
!CUDA_VISIBLE_DEVICES=0
!./build.sh
print("Building #1 done.")

# Wait again. ~5 minutes.
# Building DAIN PyTorch correlation package.
%cd /content/MVIMP/third_party/DAIN/PWCNet/correlation_package_pytorch1_0
!CUDA_VISIBLE_DEVICES=0
!./build.sh
print("Building #2 done.")

# Download pre-trained model
%cd /content/MVIMP/third_party/DAIN
!mkdir model_weights
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth
!CUDA_VISIBLE_DEVICES=0
!sudo apt-get install imagemagick imagemagick-doc
!sudo apt install zip unzip

audio_notification()

### Update OpenCV

In [0]:
import cv2
cv2.__version__

In [0]:
# Maybe not needed anymore, will have to check later.
!pip3 install --upgrade opencv-python==4.2.0.34

### Connect Google Drive

In [0]:
# Connect Google Drive
import os
from google.colab import drive
drive.mount('/content/gdrive')
directoryi = '/content/gdrive/My Drive/DAIN/Input'
directoryo = '/content/gdrive/My Drive/DAIN/Output'

if not os.path.exists(directoryi):
    os.makedirs(directoryi)
if not os.path.exists(directoryo):
    os.makedirs(directoryo)

print('Google Drive connected.')

## **Using DAIN**

### Configuration

In [0]:
# Configuration

# >Input File
VIDEO_FILE = 'test.mkv'

# >Time Step 
# Set the frame multiplier, 0.5 corresponds to 2X, 0.25 corresponds to 4X, and 0.125 corresponds to 8X.
TIMESTEP = 0.5


# >High Resolution 
# 16GB VRAM: Can handle 720p with HIGH_RES turned off.
# 8GB VRAM: Can handle 480p with HIGH_RES turned off.

# Default is False.
# Set to True to split a single frame into 4 blocks and process them separately in order to reduce GPU memory usage.
HIGH_RES = True

### Load Input Files

In [0]:
# Copy input file from Google Drive into VM ( "DAIN/Input" folder needs to be created in your Google Drive )

import shutil
import os
shutil.copy(os.path.join('/content/gdrive/My Drive/DAIN/Input', VIDEO_FILE), '/content/MVIMP/Data/Input')
audio_notification()

### Interpolate

In [0]:
# Output video file will be created in your Google Drive folder: DAIN/Output

import os
os.chdir('/content/MVIMP')

!python3 inference_dain.py --input_video {video_file} --time_step {TIMESTEP} --high_resolution {HIGH_RES}
audio_notification()